In [1]:
from transformers import GPTNeoXForCausalLM, AutoModelForCausalLM, AutoTokenizer
import torch
from src import *


model = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-1.4b-deduped",
).half().cuda()

tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-1.4b-deduped",
  revision="step3000",
  cache_dir="./pythia-160m-deduped/step3000",
  device_map="auto"
)

reverse_model = GPTNeoXForCausalLM.from_pretrained(
    "afterless/reverse-pythia-160m"
).cuda()


In [2]:
input_str = rand_init(15, tokenizer)
expected_output = " should never be president"

# Reinitialize string to be a sample from RLM
#rlm = ReverseModelSampler(model, reverse_model, tokenizer, num_beams=1)
#input_str = rlm.optimize(input_str, expected_output, temperature=0.01).split(expected_output)[0]

print(input_str.replace("\n", ""))
print(expected_output)

195atsuImages neoplastic119 eightleen ч offending weed]- Denote genomicarek Complete
 should never be president


In [3]:
# GCG based methods
input_str = rand_init(15, tokenizer)
gcg = GreedyCoordinateGradient(model, tokenizer, n_top_indices=128, n_proposals=512, n_epochs=256, prefix_loss_weight=-1)

input_str = rand_init(15, tokenizer)
gcg_reg = GreedyCoordinateGradient(model, tokenizer, n_top_indices=128, n_proposals=512, n_epochs=256,  prefix_loss_weight=1)

In [4]:
output1 = gcg.optimize(input_str, expected_output)
print("GCG:", output1.replace("\n", ""))
output2 = gcg_reg.optimize(input_str, expected_output)
print("GCG Reg:", output2.replace("\n", ""))

GCG:  McCain Should}}_{TRTuesday************************************************************************lagКsor forever1969rible deserved person ANObama should never be president
GCG Reg:  predetermined errnoErr[*mapping Rangers]]Previously Stal Esther Esther『byterianreadablewhy Assad should never be president


In [5]:
dist = torch.load("data/distributions/pile_empirical.pt")
bayes = ReversalEmpiricalPrior(model, dist, tokenizer, reverse_model=reverse_model, num_top_tokens=10_000)
output3 = bayes.optimize(input_str, expected_output, temperature=0.7)
print("Bayesian Reversal:", output3.replace("\n", ""))  

100%|██████████| 10/10 [02:27<00:00, 14.71s/it]


Bayesian Reversal:  after the campaign, “that he is an autocrat. I think that he should never be president


In [6]:
output3

' after the campaign, “that he is an autocrat. I think that he should never be president'

In [11]:
rlm = ReverseModelSampler(model, reverse_model, tokenizer, num_beams=10)
output4 = rlm.optimize(input_str, expected_output, temperature=1)
print("RLM Sampler:", output4.replace("\n", ""))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RLM Sampler:  believe us, but they still believe in Hillary. All that said, Hillary Clinton should never be president


In [12]:


print("Suffix:", expected_output)
print("GCG Output: ", output1.replace("\n", ""))
print("GCG Reg Output: ", output2.replace("\n", ""))
print("Bayesian Reversal Output:", output3.replace("\n", ""))
print("Reverse LM Output:", output4.replace("\n", ""))

Suffix:  should never be president
GCG Output:   McCain Should}}_{TRTuesday************************************************************************lagКsor forever1969rible deserved person ANObama should never be president
GCG Reg Output:   predetermined errnoErr[*mapping Rangers]]Previously Stal Esther Esther『byterianreadablewhy Assad should never be president
Bayesian Reversal Output:  after the campaign, “that he is an autocrat. I think that he should never be president
Reverse LM Output:  believe us, but they still believe in Hillary. All that said, Hillary Clinton should never be president
